In [12]:
%load_ext autoreload
%autoreload 2

In [72]:
from src.load_data import load_data
from sklearn.preprocessing import StandardScaler
import jax
import jax.numpy as jnp
from src.svm import MultiClassKernelSVM
from src.kmeans import SphericalKMeans
import pandas as pd
from typing import Tuple
from src.kernels import RBF
import time
import pickle

In [3]:
data_folder = "data"
Xtr_flat, Ytr_flat, Xte_flat = load_data(data_folder, reshape=False)
print(f"Xtr {Xtr_flat.shape}; Ytr {Ytr_flat.shape}; Xte {Xte_flat.shape}")

Xtr, Ytr, Xte = load_data(data_folder, reshape=True)
print(f"Xtr {Xtr.shape}; Ytr {Ytr.shape}; Xte {Xte.shape}")

Xtr (5000, 3072); Ytr (5000,); Xte (2000, 3072)
Xtr (5000, 32, 32, 3); Ytr (5000,); Xte (2000, 32, 32, 3)


In [4]:
print(f"Xtr mean {Xtr_flat.mean(axis=0)} ; var {Xtr_flat.var(axis=0)}")

# should we z-scored the data ?
scaler = StandardScaler()
Xtr_flat = scaler.fit_transform(Xtr_flat)
print(f"Xtr mean {Xtr_flat.mean(axis=0)} ; var {Xtr_flat.var(axis=0)}")

Xtr mean [-2.47982934e-04 -1.74261584e-03 -9.22621259e-05 ...  2.58250453e-03
  2.19476255e-03  3.04047441e-03] ; var [0.00167989 0.00118951 0.00118582 ... 0.00124412 0.00126839 0.00177964]
Xtr mean [-2.84217094e-18 -7.10542736e-18 -1.84741111e-17 ... -1.84741111e-17
 -8.52651283e-18 -1.24344979e-17] ; var [1. 1. 1. ... 1. 1. 1.]


### Kernel SVM

In [9]:
n_classes = 10
ntrain = 500

In [46]:
# from kernelchallenge.classifiers.svm import rbf_kernel

In [10]:
# my_svm = MultiClassSVM(num_classes=n_classes, kernel_func=rbf_kernel, c=1)
# my_svm.fit(Xtr_flat[:ntrain], Ytr_flat[:ntrain])
# pred = my_svm.predict(Xtr_flat[ntrain:])

# print(f"Accuracy {jnp.mean(pred == Ytr_flat[ntrain:])}")

In [6]:
from src.kernels import RBF

In [14]:
sigma = jnp.sqrt(Xtr_flat.shape[1])
c = 1
kernel_func = RBF(sigma=sigma)
my_svm = MultiClassKernelSVM(num_classes=n_classes, kernel_func=kernel_func, c=c)
my_svm.fit(Xtr_flat[:ntrain], Ytr_flat[:ntrain])
pred = my_svm.predict(Xtr_flat[ntrain:])

print(f"Accuracy {jnp.mean(pred == Ytr_flat[ntrain:])}")

Function fit took 4.92 seconds
Function predict took 43.04 seconds
Accuracy 0.14733333885669708


In [16]:
ntrain = Xtr_flat.shape[0]
kernel_func = RBF(sigma=sigma)
my_svm = MultiClassKernelSVM(num_classes=n_classes, kernel_func=kernel_func, c=c)
my_svm.fit(Xtr_flat[:ntrain], Ytr_flat[:ntrain])

Function fit took 421.93 seconds


In [ ]:
Xte = scaler.transform(Xte_flat)
Yte = my_svm.predict(Xte)
Yte = {"Prediction": Yte}
dataframe = pd.DataFrame(Yte)
dataframe.index += 1

In [21]:
dataframe.to_csv(f"{data_folder}/Yte.csv", index_label="Id")

In [32]:
# TEST ON ANOTHER DATASET
# from sklearn import datasets
# from sklearn import preprocessing
# from absl import flags


# flags.DEFINE_float("tol", 1e-5, "Tolerance of solvers.")
# flags.DEFINE_float("l2reg", 1000., "Regularization parameter. Must be positive.")
# flags.DEFINE_integer("num_samples", 20, "Size of train set.")
# flags.DEFINE_integer("num_features", 5, "Features dimension.")
# flags.DEFINE_integer("num_classes", 3, "Number of classes.")
# flags.DEFINE_bool("verbose", False, "Verbosity.")
# FLAGS = flags.FLAGS

# num_samples = 400
# num_features = 5
# num_classes = 3

# X, y = datasets.make_classification(n_samples=num_samples, n_features=num_features,
#                                       n_informative=3, n_classes=num_classes, random_state=0)
# X = preprocessing.Normalizer().fit_transform(X)
# Y = preprocessing.LabelBinarizer().fit_transform(y)
# Y = jnp.array(Y)

In [34]:
# ntrain = 300
# my_svm = MultiClassSVM(num_classes=n_classes, kernel_func=rbf_kernel, c=1)
# my_svm.fit(X[:ntrain], y[:ntrain])
# pred = my_svm.predict(X[ntrain:])

# print(f"Accuracy {jnp.mean(pred == y[ntrain:])}")

Function fit took 1.88 seconds
Function predict took 0.20 seconds
Accuracy 0.7899999618530273


# CKN 

In [45]:
f = jax.random.normal(key, (3,))
print(f)
f = f.at[f<1].set(0)
print(f)

[ 1.8160863  -0.48262316  0.33988908]
[1.8160863 0.        0.       ]


In [73]:
def get_ovelapping_patch_idx(h : int, w:int, patch_size: int =3):
        """  """
        xp, yp = jnp.meshgrid(jnp.arange(w-patch_size+1), jnp.arange(h-patch_size+1))
        x, y = jnp.meshgrid(jnp.arange(patch_size), jnp.arange(patch_size))
        X = x[None, None, ...] + xp[..., None, None]
        X = X.reshape(-1, patch_size, patch_size)
        Y = y[None, None, ...] + yp[..., None, None]
        Y = Y.reshape(-1, patch_size, patch_size)

        return (Y, X)

def normalize_row(X : jnp.ndarray) -> Tuple[jnp.ndarray, jnp.ndarray]:
     norm_rows = jnp.linalg.norm(X, axis=1, ord=2, keepdims=True)
     return X / (norm_rows + 1e-5), norm_rows

def kappa(x : jnp.ndarray, alpha : float):
     """ gaussian function """
     return jnp.exp(alpha * (x - 1))

def linear_gaussian_pooling(map : jnp.ndarray, beta : float = 1, sampling_factor : int = 2):
    h, w, c = map.shape
    hpool = h // sampling_factor
    conv_pool = jnp.zeros((hpool, hpool, c))
    z = jnp.stack(jnp.meshgrid(jnp.arange(h), jnp.arange(w), indexing='ij'), axis=2)
    for i in range(0, hpool):
        for j in range(0, hpool):
            pixel = jnp.array([i*sampling_factor, j*sampling_factor])
            conv_pool = conv_pool.at[i,j].set(jnp.sum(map * jnp.exp( - beta *  jnp.sum((pixel[None, None, :] - z)**2 , axis=2))[..., None], axis = (0, 1)))

    return conv_pool


class ConvKN:
    """ CKN Layer """

    def __init__(self, patch_size:int, out_channels : int, sampling_factor : int = 2,
                 n_patch_per_img_for_kmean : int = 10) -> None:
        self.patch_size = patch_size
        self.patch_idx = None
        self.patch_pad_idx = None
        self.Z = None
        self.out_channels = out_channels
        self.sampling_factor = sampling_factor
        self.spherical_kmeans = SphericalKMeans(nb_clusters=self.out_channels, max_iter=1000)
        self.n_patch_per_img_for_kmean = n_patch_per_img_for_kmean

    def extract_patches_2d(self, img : jnp.ndarray, inference : bool = False):
        if inference :
            if self.patch_pad_idx is None:
                h, w, _ = img.shape
                self.patch_pad_idx = get_ovelapping_patch_idx(h, w, self.patch_size)
            return img[self.patch_pad_idx]
        else :
            if self.patch_idx is None :
                # to do once
                h, w, _ = img.shape
                self.patch_idx = get_ovelapping_patch_idx(h, w, self.patch_size)

            return img[self.patch_idx]
    
    def train(self, key, input_maps):
        
        # centering the patch for the layer 0 ?
        
        # extract random patches 
        batch_size, _, _, in_channels = input_maps.shape
        X = jnp.empty((0, in_channels*self.patch_size**2))
        for i in range(batch_size):
            patches = self.extract_patches_2d(input_maps[i])
            patches = patches.reshape(-1, in_channels*self.patch_size**2)
            
            # remove constant patches
            non_cst_idx = jnp.any(patches != patches[:, [0]], axis=1) # to remove
            n_patch = min(non_cst_idx.sum(), self.n_patch_per_img_for_kmean)
            # n_patch = min(patches.shape[0], self.n_patch_per_img_for_kmean)
            key, _ = jax.random.split(key)
            X = jnp.vstack((X, 
                           jax.random.choice(key, patches[non_cst_idx], shape=(n_patch,), replace=False, axis=0))
            )
            
        X = jax.random.permutation(key, X, axis=0)
        # normalize row applying kmeans
        X, _ = normalize_row(X) 
        print(f'For training kmeans, X shape {X.shape}')
        self.Z, _ = self.spherical_kmeans.fit(X, init_centroids=self.Z) # centroids are normalized
        
        # compute linear weights
        mat = kappa(self.Z.dot(self.Z.T), alpha = 1/ .5**2) # out_channels x out_channels
        D, U = jnp.linalg.eigh(mat)
        D = D.at[D<1e-6].set(1e-6)
        inv_sqrt_D = jnp.diag(D  ** (-0.5))
        self.W = U.dot(inv_sqrt_D.dot(U.T))         
        

    def __call__(self, input_maps : jnp.ndarray):
        """
        Suppose images are squarred
        :param input_maps : array of size (B, H, W, C) B is batch size, C is channel size (e.g. 3)
        """

        if self.Z is None:
             raise Warning('Filters Z have to be initialized or learned. Call .train() befor evaluating model')

        batch_size, h, w, in_channels = input_maps.shape

        for i in range(batch_size):
            # add padding, so that there is a patch for each pixel in input map
            pad = (self.patch_size - 1)
            p = pad // 2
            image = jnp.zeros((h+pad, w+pad, in_channels))
            image = image.at[p:p+h, p:p+w, :].set(input_maps[i])

            patches = self.extract_patches_2d(image, inference=True)
            patches = patches.reshape(-1, in_channels*self.patch_size**2)

            normalized_x, norm_x = normalize_row(patches)
    
            out_map = norm_x * kappa( normalized_x.dot(self.Z.T), alpha=1/(.25)).dot(self.W)
            out_map = out_map.reshape(h, w, self.out_channels)
            # linear pooling
            beta = jnp.square(jnp.sqrt(2)/self.sampling_factor) # jnp.square(1 / (h*self.sampling_factor))

            conv_pool = linear_gaussian_pooling(out_map, beta=beta,
                                                sampling_factor=self.sampling_factor)
            if i == 0:
                out_maps = conv_pool[None, ...]
            else :
                out_maps = jnp.vstack((out_maps,
                                    conv_pool[None, ...]))
                
            if jnp.isnan(out_maps).any():
                print('i', i)
                raise Warning(f'Output map has {jnp.isnan(out_maps).sum()} NaN values')
            
        return out_maps
    

class ModelCKN:
    def __init__(self, patch_sizes : list,
                 out_channels : list,
                 subsampling_factors : list,
                 n_patch_per_img_for_kmean : int = 20):
        self.n_layers = len(out_channels)
        self.layers = []
        for i in range(self.n_layers):
            self.layers.append( ConvKN(patch_size=patch_sizes[i], 
                                       out_channels=out_channels[i],
                                       sampling_factor=subsampling_factors[i],
                                       n_patch_per_img_for_kmean = n_patch_per_img_for_kmean)
                                       )
            
    def train(self, key, input_maps, batch_size=2000):
        keys = [key]
        

        N = input_maps.shape[0]
        train_idx = jax.random.choice(key, jnp.arange(N), (N//batch_size, batch_size), replace=False)
        for b in range(len(train_idx)):
            input_maps_layer = input_maps[train_idx[b]]
            keys = jax.random.split(keys[-1], self.n_layers)
            print('b', b)
            for i in range(self.n_layers):
                self.layers[i].train(keys[i], input_maps_layer)
                input_maps_layer = self.layers[i](input_maps_layer)
                print(f'Layer {i}: out max {input_maps_layer.max()} out min {input_maps_layer.min()} Z max {self.layers[i].Z.max()} W max {self.layers[i].W.max()}')
                print(f'Output map shape {input_maps_layer.shape}')

    def __call__(self, input_maps):
    
        for i in range(self.n_layers):
            input_maps = self.layers[i](input_maps)

        return input_maps

In [4]:
data_folder = "data"
Xtr, Ytr, Xte = load_data(data_folder, reshape=True)
print(f"Xtr {Xtr.shape}; Ytr {Ytr.shape}; Xte {Xte.shape}")

Xtr (5000, 32, 32, 3); Ytr (5000,); Xte (2000, 32, 32, 3)


In [9]:
# should we z-scored the data ?
# Xtr = Xtr - Xtr.mean(axis=0)
# Xtr = Xtr/jnp.sqrt(jnp.mean(Xtr**2, axis=0, keepdims=True))
# print(f"Xtr mean {Xtr.mean(axis=0).mean()} ; var {Xtr.var(axis=0).mean()}")

Xtr mean -2.257679460049289e-11 ; var 1.0


In [ ]:
key = jax.random.PRNGKey(6)
model = ConvKN(patch_size=2, out_channels=64, sampling_factor=2, n_patch_per_img_for_kmean=20)
model.train(key, Xtr[:20])
out = model(Xtr[:20])
jnp.isnan(out).any()

In [62]:
key = jax.random.PRNGKey(9)
myCKN = ModelCKN(patch_sizes=[2, 2],
                 out_channels=[64, 128],
                 subsampling_factors=[2, 2],
                 n_patch_per_img_for_kmean=20)
key = jax.random.PRNGKey(0)
t0 = time.time()
myCKN.train(key, Xtr[:4000], batch_size=1000)
t1 = time.time()

print(f"Training myCKN took {(t1-t0)/60:.1} min ")

b 0
For training kmeans, X shape (2000, 12)
Spherical Kmean main loop starts
Spherical K means at iter 5 :  mean cosine similarity 0.8253
Spherical K means at iter 10 :  mean cosine similarity 0.8296
Spherical K means at iter 15 :  mean cosine similarity 0.8302
Spherical K means at iter 20 :  mean cosine similarity 0.8305
Spherical K means at iter 25 :  mean cosine similarity 0.8306
End of main loop
Layer 0: out max 0.744384229183197 out min -0.03800946846604347 Z max 0.7713486552238464 W max 1.5558363199234009
Output map shape (100, 16, 16, 64)
For training kmeans, X shape (2000, 256)
Spherical Kmean main loop starts
Spherical K means at iter 5 :  mean cosine similarity 0.8389
Spherical K means at iter 10 :  mean cosine similarity 0.8399
End of main loop
Layer 1: out max 2.026366949081421 out min -0.10908164829015732 Z max 0.4979182779788971 W max 2.0815765857696533
Output map shape (100, 8, 8, 128)
b 1
For training kmeans, X shape (2000, 12)
Spherical Kmean main loop starts
Spherical

In [66]:
with open('myckn1.pkl', 'wb') as f:
    pickle.dump(myCKN, f)

In [74]:
t0 = time.time()
out = myCKN(Xtr[:4000])
t1 = time.time()
print(out.min(), out.max())
print(f'Evaluate myCKN took {(t1-t0)/60:.1} min')

In [67]:
# z-score output
print(out.shape)
out = out.reshape(out.shape[0], -1)
print(out.shape)
X = out - out.mean(axis=0)[None, :]
X = X / jnp.sqrt( jnp.mean( X**2, axis=0) )[None, :]
print('Input shape', X.shape)

(1000, 8, 8, 128)
(1000, 8192)
Input shape (1000, 8192)


In [71]:
ntrain = 900
kernel_func = RBF(sigma=jnp.sqrt(X.shape[1]))
my_svm = MultiClassKernelSVM(num_classes=10, kernel_func=kernel_func, c=1)
my_svm.fit(X[:ntrain], Ytr[:ntrain])
preds = my_svm.predict(X[ntrain:])

print('Acc', jnp.mean(preds == Ytr[ntrain:1000]))

Function fit took 10.62 seconds
Function predict took 0.48 seconds
Acc 0.47
